In [1]:
import pandas as pd


In [3]:
file_path = '/content/combined_headlines_new.csv'

data = pd.read_csv(file_path)
print(data.head())

         date                                          headlines
0  2020-07-17  Jim Cramer : A better way to invest in the Cov...
1  2020-07-17    Cramer's lightning round : I would own Teradyne
2  2020-07-17  Cramer's week ahead : Big week for earnings , ...
3  2020-07-17  IQ Capital CEO Keith Bliss says tech and healt...
4  2020-07-16  Wall Street delivered the 'kind of pullback I'...


In [5]:
def clean_text(text):
  text = text.lower()
  text = text.replace('\n', ' ')
  text = text.replace('[^a-zA-Z]', ' ')
  return text

In [6]:
data['cleaned_headlines'] = data['headlines'].apply(clean_text)
print(data.head())

         date                                          headlines  \
0  2020-07-17  Jim Cramer : A better way to invest in the Cov...   
1  2020-07-17    Cramer's lightning round : I would own Teradyne   
2  2020-07-17  Cramer's week ahead : Big week for earnings , ...   
3  2020-07-17  IQ Capital CEO Keith Bliss says tech and healt...   
4  2020-07-16  Wall Street delivered the 'kind of pullback I'...   

                                   cleaned_headlines  
0  jim cramer : a better way to invest in the cov...  
1    cramer's lightning round : i would own teradyne  
2  cramer's week ahead : big week for earnings , ...  
3  iq capital ceo keith bliss says tech and healt...  
4  wall street delivered the 'kind of pullback i'...  


In [7]:
import yfinance as yf

In [8]:
spy_data = yf.download('SPY', start= '2020-01-01', end='2023-01-01')
spy_data.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [9]:
spy_data = spy_data[['Date', 'Adj Close']]
spy_data.rename(columns= {'Adj Close': 'spy_price'}, inplace=True)

<ipython-input-9-714537a9927d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spy_data.rename(columns= {'Adj Close': 'spy_price'}, inplace=True)


In [10]:
data['Date'] = pd.to_datetime(data['date'])
merged_data = pd.merge(data, spy_data, how='left', left_on= 'Date', right_on='Date')

In [11]:
print(merged_data.head())

         date                                          headlines  \
0  2020-07-17  Jim Cramer : A better way to invest in the Cov...   
1  2020-07-17    Cramer's lightning round : I would own Teradyne   
2  2020-07-17  Cramer's week ahead : Big week for earnings , ...   
3  2020-07-17  IQ Capital CEO Keith Bliss says tech and healt...   
4  2020-07-16  Wall Street delivered the 'kind of pullback I'...   

                                   cleaned_headlines       Date   spy_price  
0  jim cramer : a better way to invest in the cov... 2020-07-17  303.290314  
1    cramer's lightning round : i would own teradyne 2020-07-17  303.290314  
2  cramer's week ahead : big week for earnings , ... 2020-07-17  303.290314  
3  iq capital ceo keith bliss says tech and healt... 2020-07-17  303.290314  
4  wall street delivered the 'kind of pullback i'... 2020-07-16  302.413696  


In [15]:
merged_data['price_change'] = merged_data['spy_price'].pct_change()
merged_data['target'] = (merged_data['price_change'] > 0).astype(int)
merged_data.dropna(subset=['price_change'], inplace=True)

<ipython-input-15-7ef1ea9759e2>:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  merged_data['price_change'] = merged_data['spy_price'].pct_change()


In [16]:
print(merged_data[['Date', 'spy_price', 'price_change', 'target']].head())

        Date   spy_price  price_change  target
1 2020-07-17  303.290314       0.00000       0
2 2020-07-17  303.290314       0.00000       0
3 2020-07-17  303.290314       0.00000       0
4 2020-07-16  302.413696      -0.00289       0
5 2020-07-16  302.413696       0.00000       0


In [17]:
#Finalize the Dataset

final_data = merged_data[['cleaned_headlines', 'target']]

In [18]:
print(final_data['target'].value_counts())

target
0    53141
1      188
Name: count, dtype: int64


In [26]:
print(final_data.head())

                                   cleaned_headlines  target
1    cramer's lightning round : i would own teradyne       0
2  cramer's week ahead : big week for earnings , ...       0
3  iq capital ceo keith bliss says tech and healt...       0
4  wall street delivered the 'kind of pullback i'...       0
5  cramer's lightning round : i would just stay l...       0
